In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization, Activation, concatenate, GlobalAveragePooling2D, AveragePooling2D, GlobalMaxPooling2D
from keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#load satellite image classification
dir = '/kaggle/input/satellite-image-classification/data'


cloudy = os.path.join(dir, 'cloudy')
desert = os.path.join(dir, 'desert')
grean_area = os.path.join(dir, 'green_area')
water = os.path.join(dir, 'water')

categories = ['cloudy', 'desert', 'green_area', 'water']

#load images
cloudy_images = os.listdir(cloudy)
desert_images = os.listdir(desert)
grean_area_images = os.listdir(grean_area)
water_images = os.listdir(water)



#Array to store images
x = []
#Array to store labels
y = []

image_size = (224,224)

In [4]:
from skimage.transform import resize
#load cloudy images
for i in range(len(cloudy_images)):
    path = os.path.join(cloudy, cloudy_images[i])
    img = plt.imread(path)
    img_resized = resize(img, image_size+(3,))
    x.append(img_resized)
    y.append(0)

#load desert images
for i in range(len(desert_images)):
    path = os.path.join(desert, desert_images[i])
    img = plt.imread(path)
    img_resized = resize(img, image_size+(3,))
    x.append(img_resized)
    y.append(1)

#load grean_area images
for i in range(len(grean_area_images)):
    path = os.path.join(grean_area, grean_area_images[i])
    img = plt.imread(path)
    img_resized = resize(img, image_size+(3,))
    x.append(img_resized)
    y.append(2)

#load water images
for i in range(len(water_images)):
    path = os.path.join(water, water_images[i])
    img = plt.imread(path)
    img_resized = resize(img, image_size+(3,))
    x.append(img_resized)
    y.append(3)
    
#convert to numpy array
x = np.array(x)
y = np.array(y)

#shuffle the data
np.random.shuffle(x)
np.random.shuffle(y)

In [5]:
from sklearn.model_selection import train_test_split
#split the data into train and test 20%
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

#one hot encoding
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

#load the MobileNetV2 model
from keras.applications import mobilenet_v2

base_model = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

#freeze the layers
for layer in base_model.layers:
    layer.trainable = False

#add new layers 
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())

model.summary()

(4504, 224, 224, 3) (1127, 224, 224, 3) (4504,) (1127,)
9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
Total params: 2257984 (8.61 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [6]:
#feuture extraction
feture_train = model.predict(x_train)
feture_test = model.predict(x_test)

#flatten extracted feture
feature_train = feture_train.reshape(feture_train.shape[0], -1)
feature_test = feture_test.reshape(feture_test.shape[0], -1)

#flatten labels
y_train_flattened = y_train_onehot[:, 1]
y_test_flattened = y_test_onehot[:, 1]

print("Before train-test split:", x.shape, y.shape)
print("After train-test split:", x_train.shape, y_train.shape, x_test.shape, y_test.shape)
print("Flattened shapes:", y_train_flattened.shape, y_test_flattened.shape)

36/36 [==============================] - 2s 46ms/step
Before train-test split: (5631, 224, 224, 3) (5631,)
After train-test split: (4504, 224, 224, 3) (4504,) (1127, 224, 224, 3) (1127,)
Flattened shapes: (4504,) (1127,)


In [7]:
from sklearn.svm import SVC
# Genetic Algorithm Parameters
POP_SIZE = 10
GENES = feature_train.shape[1]
GENERATIONS = 36
MUTATION_RATE = 0.01
CROSSOVER_RATE = 0.7
AVERAGE_ACC = [] # Average accuracy of each generation

# Initialize population
def initialize_population(population_size, genes):
    return np.random.choice([0, 1], size=(population_size, genes))

# Evaluate fitness using nn classifier
def evaluate_fitness(population, X_train, X_test, y_train, y_test):
    fitness_values = []
    for chromosome in population:
        selected_features = np.where(chromosome == 1)[0]
        if len(selected_features) == 0:
            fitness_values.append(0)
            continue
        X_train_selected = X_train[:, selected_features]
        X_test_selected = X_test[:, selected_features]
        model = SVC(kernel='poly', C=1)
        model.fit(X_train_selected, y_train)
        test_acc = model.score(X_test_selected, y_test)
        fitness_values.append(test_acc)

    return np.array(fitness_values)

# Perform crossover
def crossover(parent1, parent2):
    crossover_point = np.random.randint(0, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

# Perform mutation
def mutate(child):
    mutation_mask = np.random.rand(len(child)) < MUTATION_RATE
    child[mutation_mask] = 1 - child[mutation_mask]  # Flip the bits
    return child

# Main Genetic Algorithm
def genetic_algorithm(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    population = initialize_population(POP_SIZE, GENES)

    for generation in range(GENERATIONS):
        fitness_values = evaluate_fitness(population, X_train, X_test, y_train, y_test)
        selected_indices = np.argsort(fitness_values)[::-1][:int(POP_SIZE * CROSSOVER_RATE)]

        new_population = []
        AVERAGE_ACC.append(np.mean(fitness_values))
        # Perform crossover and mutation
        for _ in range(0, POP_SIZE, 2):
            parent1 = population[np.random.choice(selected_indices)]
            parent2 = population[np.random.choice(selected_indices)]
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.extend([child1, child2])

        population = np.array(new_population)

    best_individual = population[np.argmax(fitness_values)]
    selected_features = np.where(best_individual == 1)[0]
    return selected_features

# usage
selected_features = genetic_algorithm(feature_train, y_train_flattened)
print("Selected Features:", selected_features)

Selected Features: [   3    9   10   11   13   14   15   20   21   23   24   25   28   29
   30   31   33   35   36   38   40   41   44   46   47   50   53   54
   55   56   58   59   60   61   62   63   66   68   69   70   71   72
   76   77   80   83   84   87   92   93   94  103  105  108  109  110
  111  113  114  115  117  118  119  120  121  122  125  127  128  130
  131  134  136  137  138  140  141  142  143  144  148  154  155  156
  157  158  159  160  162  163  164  166  167  169  172  175  176  178
  185  187  188  189  191  193  194  195  196  197  199  200  202  205
  206  207  209  212  216  220  221  222  225  227  228  229  231  232
  234  237  238  240  241  244  247  249  251  253  254  255  256  257
  262  265  267  272  276  278  279  282  284  286  287  289  293  295
  297  300  302  303  304  305  310  313  314  315  322  323  325  328
  329  330  331  335  338  339  340  342  345  348  349  352  353  355
  358  360  361  362  365  367  371  372  373  375  379  3

In [16]:
# train svm model on selected features
train_selected_features = feature_train[:, selected_features]
test_selected_features = feature_test[:, selected_features]
from sklearn.svm import SVC
model = SVC(kernel='poly', C=1)
model.fit(train_selected_features, y_train_flattened)
print('Accuracy on selected features:',  model.score(test_selected_features, y_test_flattened))

Accuracy on selected features: 0.7950310559006211
